In [ ]:
""" Aqui se presentan las funciones basicas para la extraccion de la tractografia y la extraccion de las conectividades estructurales a partir del tractograma y una parcelacion """

In [ ]:
def tractograma_determinista_PFT(fraw, fbval, fbvec, fmask, ft1):
  """
  Esta función recibe las rutas de los archivos de resonancia magnetica de difusion y retorna un archivo de streamlines extraido por el algoritmo determinista de particle filtering tractografia

  Parámetros
  ----------
  fraw : Ruta al archivo data.nii.gz donde se almacenan los datos de resonancia magnetica de difusion para cada bvec y bval
  fbval y fbvec : Rutas a los archivos bvals y bvecs sobre los que se tomaron las imagenes de resonancia magnetica de difusion
  fmask : Ruta al archivo brainmask_fs.nii.gz con la mascara de la misma dimension que data.nii.gz para depurar las imagenes de tejidos no cerebrales
  ft1 : Ruta al archivo T1w_acpc_dc_restore_brain.nii.gz del cual se generaran las mascaras de tejidos (suele tener una mayor resolucion que data.nii.gz, esta funcion luego redimensiona las mascaras)
  
  Regresa
  -------
  Retorna un objeto base para representar las streamlines en cualquier formato (trk, tck, vtk, fib, dpy)
  """
  
  #%% Cargar los datos del HCP1200, en este caso con su respectiva matriz affine y el tamaño de los voxeles en la imagen

  from dipy.io.image import load_nifti_data, load_nifti, save_nifti

  data, affine, voxel_size = load_nifti(fraw, return_voxsize=True) # Subir data de aprox 1gb es computacionalmente exigente y puede se demorado
  t1_data, t1_affine, t1_voxel_size = load_nifti(ft1, return_voxsize=True)
  mask_data, mask_affine, mask_voxel_size = load_nifti(fmask, return_voxsize=True) 

  #%% Leer los bvals y bvecs así como crear gtab.

  from dipy.core.gradients import gradient_table #, unique_bvals_tolerance, get_bval_indices, round_bvals
  from dipy.io.gradients import read_bvals_bvecs

  bvals, bvecs = read_bvals_bvecs(fbval, fbvec)
  gtab = gradient_table(bvals, bvecs)

  #%% Realizar la separación del tejido con la imagen de referencia de alta resolución T1w

  from dipy.segment.tissue import TissueClassifierHMRF

  nclass = 3 # Número de tejidos para los que obtener máscaras: csf, gm, wm
  beta = 0.1 # Suavidad del trazado de las máscaras
  hmrf = TissueClassifierHMRF()
  initial_segmentation, final_segmentation, PVE = hmrf.classify(t1_data, nclass, beta) # Toma alrededor de 22 iteraciones, puede demorarse

  # Máscaras sin redimensionar
  csf = np.where(final_segmentation == 1, 1, 0)
  gm = np.where(final_segmentation == 2, 1, 0)
  wm = np.where(final_segmentation == 3, 1, 0)

  #%% Con las máscaras, se procede a redimensionarlas al tamaño de la imagen guía para las parcelas

  from dipy.align.reslice import reslice

  ds = data.shape
  t1s = t1_data.shape
  ms = mask_data.shape

  t1_new_voxel_size = []
  mask_new_voxel_size = []

  for i in range(3):
      t1_new_voxel_size.append(t1s[i]*t1_voxel_size[i]/ds[i])
      mask_new_voxel_size.append(ms[i]*mask_voxel_size[i]/ds[i])

  # Las máscaras quedan redimensionadas a la resolución de los datos con su respectiva matriz affine
  rwm, rwm_affine = reslice(wm, t1_affine, t1_voxel_size, t1_new_voxel_size)
  rgm, rgm_affine = reslice(gm, t1_affine, t1_voxel_size, t1_new_voxel_size)
  rcsf, rcsf_affine = reslice(csf, t1_affine, t1_voxel_size, t1_new_voxel_size)

  # Se redimensiona la máscara para aplicarla a los datos originales
  rmask_data, rmask_affine = reslice(mask_data, mask_affine, mask_voxel_size, mask_new_voxel_size)

  #%% Se aplica la máscara redimensionada a los datos 

  from dipy.segment.mask import applymask

  # Nuevos datos de referencia, sin los tejidos ajenos al cerebro
  datamask = applymask(data, rmask_data)
  datamask_affine = affine
  datamask_voxel_size = voxel_size

  del data,  # Para vaciar la RAM y permitir el no se agoten los recursos del google colab

  #%% Se define el modelo tensorial para aplicar a los voxeles, que recibe como parámetro gtab, es decir, la informacion de los gradientes y constantes importantes relevantes para hallar el tensor

  import dipy.reconst.dti as dti

  dti_model = dti.TensorModel(gtab)

  #%% Generar los picos en cada voxel para la tractografáa 

  # Se carga la esfera de 362 puntos sobre la que los autovectores asociados a los tensores en cada voxel se proyectaran
  from dipy.data import get_sphere
  sphere = get_sphere('symmetric362')

  from dipy.direction import peaks_from_model

  # Se determinan las direcciones o picos de máxima difusión para cada voxel, esta parte puede tardar un rato
  peak_indices = peaks_from_model(
      model=dti_model, data=datamask, sphere=sphere, relative_peak_threshold=.2,
      min_separation_angle=25, mask=rmask_data, npeaks=2)

  #%% Generar los criterios para empezar y parar los tractos nerviosos 
  from dipy.tracking import utils
  from dipy.tracking.stopping_criterion import CmcStoppingCriterion

  voxel_size_mask = np.average(t1_new_voxel_size)
  step_size = 0.5

  # Se definen las semillas de la tractografía con la máscara de materia blanca
  seeds = utils.seeds_from_mask(rwm, datamask_affine, density=1)

  # Se define el Stopping Criterion con las máscaras de los tres tejidos rwm, rgm, rcsf
  cmc_criterion = CmcStoppingCriterion.from_pve(rwm,
                                                rgm,
                                                rcsf,
                                                step_size=step_size,
                                                average_voxel_size=voxel_size_mask)

  #%%  Se define la Particle Filtering Tractography, enviando como parámetro las semillas, el StoppingCriterion y los picos para cada voxel

  from dipy.tracking.local_tracking import (LocalTracking,ParticleFilteringTracking)

  pft_streamline_generator = ParticleFilteringTracking(peak_indices,
                                                      cmc_criterion,
                                                      seeds,
                                                      datamask_affine,
                                                      max_cross=1,
                                                      step_size=step_size,
                                                      maxlen=1000,
                                                      pft_back_tracking_dist=2,
                                                      pft_front_tracking_dist=1,
                                                      particle_count=15,
                                                      return_all=False)
  
  #%% Definido el tipo de tractografía, con Streamlines se generan las curvas como se especificó arriba

  from dipy.tracking.streamline import Streamlines

  pft_streamlines = Streamlines(pft_streamline_generator)

  #%% Guardar tractograma en archivo trk con el formato canónico RAS+ en mm

  from dipy.io.stateful_tractogram import Space, StatefulTractogram
  from dipy.io.streamline import save_tractogram

  # Para generar el tractograma pft_sft es necesario enviar como parámetro el archivo NIFTI de donde se obtuvieron los tractos
  data_img = nib.load(fraw)

  # Se genera el tractograma en el espacio RASMM
  pft_sft = StatefulTractogram(pft_streamlines, data_img, Space.RASMM)

  # Se guarda el tractograma en formato trk, este se puede visualizar con el programa TrackVis o cargandolo a python como se verá más adelante

  return pft_sft


In [ ]:
def conectividades_estructurales(patient, streamlines, parcelacion, par_data, par_affine, dpar):
  """
  Esta función recibe los datos del paciente, sus streamlines, y los datos de la parcelacion a usar, guardando las matrices estructurales producidas

  Parámetros
  ----------
  patient, streamlines : Cadena de texto (string) que identifica al paciente (patient), junto con sus streamlines
  streamlines : Streamlines asociadas al sujeto
  parcelacion, par_data, par_affine : Cadena de texto (string) que identifica la parcelacion usada, junto con su respectivo archivo par_data donde se almacenan los labels que identifican las parcelas y la transformacion par_affine
  dpar : Ruta a la carpeta donde se guardaran las matrices estructurales generadas

  Regresa
  -------
  Retorna tres archivos .npy que contienen la diferentes matrices estructurales normalizadas al valor de la entrada maxima, en terminos del conteo de tractos o la longitud promedio de los tractos entre dos parcelas

  sum : Cada entrada representa el numero de tractos que conectan dos parcelas
  len : Cada entrada representa la longitud promedio de los tractos que conectan dos parcelas
  invlen : Cada entrada representa el inverso de la longitud promedio de los tractos que conectan dos parcelas
  """

  #%% Con los datos del tractograma se procede a crear la matriz con la parcela elegida anteriormente, esto se hará por medio el conteo de tractos que van de una parcela a otra, definiendo un link pesado
  from dipy.tracking import utils

  # Se extrae la matriz de conectividad M pasando la matriz affine y la imagen de la parcelación correspondiente
  preM, grouping = utils.connectivity_matrix(streamlines, par_affine, label_volume=par_data.astype(int), return_mapping=True, mapping_as_streamlines=False)

  # Redimensionar la matriz estructural sin el nodo empty con label cero
  M = preM[1:len(preM),1:len(preM)]

  #%% En esta parte se utiliza el diccionario grouping para generar la matriz structural proporcional al promedio de la longitud de los tractos que unen una parcela con la otra

  # Crea una lista donde cada posicion corresponde a la longitud lenghts(i) del tracto de label i
  from dipy.tracking.utils import length

  lengths = list(length(streamlines))

  # Se crea un ndarray de ceros con el mismo tamaño de M para guardar los datos
  M_len, M_invlen = np.zeros(M.shape), np.zeros(M.shape)

  # Se recoren los posibles links para calcular en cada uno la longitud promedio de todos los tractos
  for i in range(0, M.shape[0]):
    for j in range(i, M.shape[1]):

      # index: contiene los indices para acceder a los tractos que conectan dos parcelas en el vector de longitudes
      index = grouping[i,j]
      # longitud: es el numero de tractos que conectan dos parcelas
      longitud = 1.0*len(index)
      # sum: será la variable que contendra la suma total de la longitud de los tractos que conectan dos parcelas
      sum = 0.0

      # Si no hay tractos (longitud = 0.0) entre dos parcelas el valor en la matriz será cero, de lo contrario será el promedio de las longitudes o su inversa
      if longitud == 0.0:

        # Se llena la matriz de adyacencia para el caso donde no existan tractos que conecten las dos regiones
        M_len[i,j], M_len[j,i] = 0.0, 0.0
        M_invlen[i,j], M_invlen[j,i] = 0.0, 0.0

      # Si hay tractos se proceden a calcular la matriz de longitudes 1/longitudes
      else:

        # Como se mencionó antes, sum se usa para sumar las longitudes de los tractos entre dos parcelas
        for k in index:
          sum += lengths[k]
        # Con sum y longitud se calcula el promedio de la longitudes de los tractos que conectan dos parcelas
        prom = sum/longitud
        # Se llena la matriz de adyacencia con el valor promedio y 1/promedio de la longitud
        M_len[i,j], M_len[j,i] = prom, prom
        M_invlen[i,j], M_invlen[j,i] = 1.0/prom, 1.0/prom

  #%% Se guardan las matrices normalizando con el maximo valor
  from os.path import expanduser, join

  name = 'EST' + '_sum_no_norm_' + patient + parcelacion + '.npy'
  festconnectivity = join(dpar, name)
  np.save(festconnectivity, M)

  M = M/M.max()
  name = 'EST' + '_sum_' + patient + parcelacion + '.npy'
  festconnectivity = join(dpar, name)
  np.save(festconnectivity, M)

  M_len = M_len/M_len.max()
  name = 'EST' + '_len_' + patient + parcelacion + '.npy'
  festconnectivity = join(dpar, name)
  np.save(festconnectivity, M_len)

  M_invlen = M_invlen/M_invlen.max()
  name = 'EST' + '_invlen_' + patient + parcelacion + '.npy'
  festconnectivity = join(dpar, name)
  np.save(festconnectivity, M_invlen)
  